<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/159_AutoGen_Pricing_Compliance_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 🤖 Autogen Mental Model

Autogen orchestrates agents through **natural multi-agent conversations**.
Each agent:

* Has a **role**, **goal**, and optionally **tools**
* Can **send messages**, **call functions**, and **respond intelligently**
* Operates more like a “colleague” in a chatroom than a static chain

### 🧠 What Makes Autogen Different?

| Feature                           | Description                                     |
| --------------------------------- | ----------------------------------------------- |
| 🗣️ **Agents talk to each other** | Instead of executing tasks, they discuss them   |
| 🧠 **Autonomy**                   | You can let them reason and decide when to stop |
| 🧰 **Function calls/tools**       | Easily attach tools, APIs, DB queries, etc.     |
| 🔄 **Flexible coordination**      | Agents negotiate, revise, escalate on their own |

---

## 💼 Use Case: Pricing Compliance Agent (Autogen)

**Conversation-style workflow:**

1. **User Proxy Agent** initiates pricing check.
2. **Data Agent** provides product pricing.
3. **Policy Agent** performs RAG on compliance rules.
4. **Compliance Agent** evaluates risk and responds.
5. Optional: **Escalation Agent** joins if needed.

---





## ✅ Example Conversation Flow

```text
DataAgent: Pricing Info: Product 123 — Base $100, Discount 40%
PolicyAgent: Using RAG to retrieve rules...
PolicyAgent: Relevant rule: Discounts over 30% require manager approval.
ComplianceAgent: This pricing requires escalation. Policy violation detected.
```

---

## ✅ Autogen Strengths for Orchestration

| Feature                 | How it Shows Up                                 |
| ----------------------- | ----------------------------------------------- |
| 🗣️ Agent Communication | Each step is an actual dialogue                 |
| 🧠 Dynamic Reasoning    | Agents can ask each other for clarification     |
| 🔧 Tool Calls           | Agents can call retrieval or DB functions       |
| 🔁 Flexibility          | Conversations adapt without rigid control logic |

---

## 🧠 When to Use Autogen

| Use Case                                                         | Fit                     |
| ---------------------------------------------------------------- | ----------------------- |
| Simulating real-world workflows (like support or approval teams) | ✅✅✅                     |
| Collaborative decision-making with role clarity                  | ✅✅                      |
| Fine-grained control over orchestration logic                    | ❌ (LangGraph is better) |
| Human-like reasoning, escalation, negotiation                    | ✅✅✅                     |

---

## 🧭 What’s Next?

You now have:

* ✅ LangChain → component-oriented
* ✅ LangGraph → graph/state orchestration
* ✅ CrewAI → team/task orchestration
* ✅ Autogen → agent conversation orchestration

Would you like:

* A **final matrix/table** comparing all 4 frameworks across orchestration dimensions?
* A **summary notebook** with links + bullet points for each implementation?
* Or help designing your own **custom orchestrator project** to build from scratch?


## 🚀 Autogen Implementation (Simplified 3-agent version)

In [ ]:
# !pip install pyautogen openai faiss-cpu langchain

from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# ============================
# 🧰 RAG utility (used by Policy Agent)
def get_compliance_rules(pricing_text: str) -> str:
    compliance_docs = [
        "Discounts over 30% require manager approval.",
        "All pricing must be auditable and linked to policy ID."
    ]
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(compliance_docs, embeddings)
    retriever = vectorstore.as_retriever()
    docs = retriever.get_relevant_documents(pricing_text)
    return "\n".join([doc.page_content for doc in docs])

# ============================
# 👤 Define Agents

# 1. User initiator
user_proxy = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    code_execution_config=False
)

# 2. Data Agent: Returns pricing info
data_agent = AssistantAgent(
    name="DataAgent",
    system_message="You are a product data assistant. Return pricing info when asked.",
)

# 3. Policy Agent: Runs RAG on compliance
policy_agent = AssistantAgent(
    name="PolicyAgent",
    system_message="You extract relevant compliance rules using semantic search. Call the RAG function to retrieve policies.",
    code_execution_config={"use_docker": False},
    function_map={
        "get_compliance_rules": get_compliance_rules
    }
)

# 4. Compliance Agent: Final decision maker
compliance_agent = AssistantAgent(
    name="ComplianceAgent",
    system_message="You evaluate pricing against compliance rules and decide if escalation is needed."
)

# ============================
# 👥 GroupChat Orchestrator

groupchat = GroupChat(
    agents=[user_proxy, data_agent, policy_agent, compliance_agent],
    messages=[],
    max_round=6
)

manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": [{"model": "gpt-4"}]})

# ============================
# 💬 Start the Conversation

user_proxy.initiate_chat(
    manager,
    message="Can someone check if Product 123 with a 40% discount complies with policy?"
)



## 🧠 What You're Doing in AutoGen

You're using **AutoGen's multi-agent chat system**:

### 🧰 Tools:

* **RAG function**: `get_compliance_rules()` uses FAISS + LangChain to do semantic retrieval on compliance policy text.

### 🧑‍🤝‍🧑 Agents:

1. **UserProxyAgent**: Kicks off the task with a message.
2. **DataAgent**: Returns pricing info.
3. **PolicyAgent**: Uses the RAG function to fetch matching policies.
4. **ComplianceAgent**: Decides whether escalation is required.

### 🤖 Orchestration:

* Agents are connected in a `GroupChat` managed by `GroupChatManager`.
* You don’t define the order — the agents “chat” and collaborate to reach the answer within `max_round=6`.

---

## 🔍 Comparison: AutoGen vs Your Agent Recipe

Let’s analyze how well AutoGen aligns with your Agent Recipe principles.

| 🧪 Agent Recipe Principle      | ✅ AutoGen Support? | Commentary                                                                               |
| ------------------------------ | ------------------ | ---------------------------------------------------------------------------------------- |
| **Modular, functional design** | ✅ Yes              | Each agent is clearly scoped and isolated                                                |
| **Avoid tight coupling**       | ✅ Yes              | Agents interact only via conversation, no direct code dependencies                       |
| **Auditable workflow**         | ⚠️ Limited         | You can log chats, but state is hidden in LLM outputs — not structured like `AgentState` |
| **Reusable components**        | ✅ Yes              | Agents can be reused with different messages or group configurations                     |
| **Statefulness**               | ⚠️ Implicit only   | No explicit state object — all context is passed through chat                            |
| **Custom routing/logic**       | ⚠️ Emergent        | Order of execution is inferred from LLM behavior, not controlled                         |
| **Tool integration**           | ✅ Strong           | You map Python functions to agents (like `get_compliance_rules`)                         |

---

## 🧠 Key Strengths of AutoGen

| Feature                         | Why It’s Good                                                           |
| ------------------------------- | ----------------------------------------------------------------------- |
| ✅ **High-level abstraction**    | Agents act like chat participants — very natural to model conversations |
| ✅ **Built-in LLM coordination** | You don’t need to orchestrate manually — it’s handled by the system     |
| ✅ **Tool calling support**      | Agents can execute Python functions (e.g., RAG) when they “think” to    |
| ✅ **Extensible**                | You can plug in code execution, memory, etc. on a per-agent basis       |

---

## ⚠️ Key Limitations vs LangGraph

| Weakness                        | Explanation                                                            |
| ------------------------------- | ---------------------------------------------------------------------- |
| ❌ **No structured state**       | No centralized `AgentState` — you can't inspect step-by-step data flow |
| ❌ **No deterministic flow**     | You can't define fixed routing logic (e.g., conditionals, loops)       |
| ❌ **Debugging is hard**         | Agent behavior is emergent — you’re at the mercy of LLM decisions      |
| ❌ **Limited testing**           | Hard to unit test agents or intermediate state                         |
| ❌ **No workflow visualization** | No graph or audit trail like LangGraph’s Mermaid support               |

---

## 🧠 Summary: Who AutoGen Is For

| Ideal For…                           | Not Ideal For…                   |
| ------------------------------------ | -------------------------------- |
| 🤝 Collaborative chat simulations    | 🧪 Deterministic agent pipelines |
| 🧪 Prototyping multi-agent behaviors | 🧰 Production orchestrators      |
| 🧠 Exploring emergent LLM behaviors  | 🔍 Auditable, testable workflows |
| 🧙 Tool-calling from LLM agents      | 🧱 Stateful data pipelines       |

---

## 🧠 Final Verdict (with Recipe Scorecard)

| Agent Framework | Modular | Loosely Coupled | Auditable | Stateful | Reusable | Testable | Recipe Alignment      |
| --------------- | ------- | --------------- | --------- | -------- | -------- | -------- | --------------------- |
| **LangGraph** ✅ | ✅       | ✅               | ✅         | ✅        | ✅        | ✅        | ⭐⭐⭐⭐⭐ Excellent       |
| **CrewAI** ⚠️   | ✅       | ⚠️              | ⚠️        | ❌        | ✅        | ⚠️       | ⭐⭐ Basic              |
| **AutoGen** ⚠️  | ✅       | ✅               | ⚠️        | ⚠️       | ✅        | ⚠️       | ⭐⭐⭐ Good but Emergent |

---

## ✅ TL;DR

> **AutoGen is a strong prototyping tool for emergent behaviors** and tool-using agents — but if you need deterministic, auditable, testable agent orchestration… LangGraph is still the king.


